In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import xarray as xr

In [2]:
metrics = xr.open_dataset("climate_change_metrics.nc")

In [3]:
def simple_significance_test(a, b, dim):
    a_abs = np.abs(a)
    b_abs = np.abs(b)
    
    a_min = a_abs.min(dim)
    a_max = a_abs.max(dim)
    b_min = b_abs.min(dim)
    b_max = b_abs.max(dim)
    
    return (a_max < b_min) | (b_max < a_min)

In [4]:
VARIABLES = ["total_precipitation_rate",  "surface_temperature", "net_surface_radiative_flux"]
REGIONS = ["land", "ocean/sea-ice"]


def metrics_table(metrics, configuration, validation_year):
    metrics = metrics.sel(validation_year=validation_year).drop("validation_year")
    stacked_metrics = metrics.stack(sample=("configuration", "time")).drop("samples")
    stacked_metrics = stacked_metrics.dropna("sample")
    
    in_five_year_window = (stacked_metrics.time.dt.year.isin(range(2018, 2023)))
    baseline_samples = (stacked_metrics.configuration == "Baseline") & in_five_year_window
    configuration_samples = (stacked_metrics.configuration == configuration) & in_five_year_window
    
    significant = simple_significance_test(
        stacked_metrics.isel(sample=configuration_samples),
        stacked_metrics.isel(sample=baseline_samples),
        "sample"
    )
    
    mean_metrics = xr.concat(
        [
            stacked_metrics.sel(sample=baseline_samples).mean("sample"),
            stacked_metrics.sel(sample=configuration_samples).mean("sample")
        ],
        dim=pd.Index(["Baseline", configuration], name="configuration")
    )
    
    unmasked_metrics = (
        mean_metrics[VARIABLES]
        .sel(region=REGIONS)
        .to_array()
        .rename("metric")
        .to_dataframe(dim_order=["variable", "metric", "region", "configuration", "climate"])
        .unstack(level=-1)
        .unstack(level=-1)
    )
    masked_metrics = (
        mean_metrics[VARIABLES]
        .where(significant[VARIABLES])
        .sel(region=REGIONS)
        .to_array()
        .rename("metric")
        .to_dataframe(dim_order=["variable", "metric", "region", "configuration", "climate"])
        .unstack(level=-1)
        .unstack(level=-1)
    )
    return mean_metrics, significant[VARIABLES], unmasked_metrics, masked_metrics

In [5]:
mean_metrics_year_one, significant_year_one, unmasked_year_one, masked_year_one = metrics_table(metrics, "ML-corrected seed 2", "Year one")
mean_metrics_year_two, significant_year_two, unmasked_year_two, masked_year_two = metrics_table(metrics, "ML-corrected seed 2", "Year two")

### Source of data in Table 2 of the manuscript

In [6]:
unmasked_year_two.round(decimals=1)

metric  \
climate                                         Unperturbed minus Minus 4 K   
configuration                                                      Baseline   
variable                   metric region                                      
total_precipitation_rate   mean   land                                 -0.1   
                                  ocean/sea-ice                         0.0   
                           rmse   land                                  0.9   
                                  ocean/sea-ice                         1.3   
                           mae    land                                  0.5   
                                  ocean/sea-ice                         0.8   
surface_temperature        mean   land                                  0.1   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  1.6   
                                  ocean/sea-ice                         0.4   
                           mae    land                                  1.1   
                                  ocean/sea-ice                         0.1   
net_surface_radiative_flux mean   land                                 -2.7   
                                  ocean/sea-ice                        -0.4   
                           rmse   land                                  9.8   
                                  ocean/sea-ice                         7.8   
                           mae    land                                  6.4   
                                  ocean/sea-ice                         5.8   

                                                                     \
climate                                                               
configuration                                   ML-corrected seed 2   
variable                   metric region                              
total_precipitation_rate   mean   land                          0.0   
                                  ocean/sea-ice                -0.1   
                           rmse   land                          0.8   
                                  ocean/sea-ice                 1.5   
                           mae    land                          0.5   
                                  ocean/sea-ice                 0.9   
surface_temperature        mean   land                         -0.3   
                                  ocean/sea-ice                 0.0   
                           rmse   land                          1.5   
                                  ocean/sea-ice                 0.4   
                           mae    land                          1.1   
                                  ocean/sea-ice                 0.1   
net_surface_radiative_flux mean   land                          0.2   
                                  ocean/sea-ice                 0.1   
                           rmse   land                          7.3   
                                  ocean/sea-ice                 8.9   
                           mae    land                          5.1   
                                  ocean/sea-ice                 6.5   

                                                                            \
climate                                         Plus 4 K minus Unperturbed   
configuration                                                     Baseline   
variable                   metric region                                     
total_precipitation_rate   mean   land                                 0.1   
                                  ocean/sea-ice                       -0.0   
                           rmse   land                                 1.0   
                                  ocean/sea-ice                        1.9   
                           mae    land                                 0.6   
                                  ocean/sea-ice                  

In [7]:
masked_year_two.round(decimals=1)

metric  \
climate                                         Unperturbed minus Minus 4 K   
configuration                                                      Baseline   
variable                   metric region                                      
total_precipitation_rate   mean   land                                  NaN   
                                  ocean/sea-ice                         0.0   
                           rmse   land                                  0.9   
                                  ocean/sea-ice                         1.3   
                           mae    land                                  0.5   
                                  ocean/sea-ice                         NaN   
surface_temperature        mean   land                                  NaN   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  NaN   
                                  ocean/sea-ice                         NaN   
                           mae    land                                  NaN   
                                  ocean/sea-ice                         NaN   
net_surface_radiative_flux mean   land                                 -2.7   
                                  ocean/sea-ice                         NaN   
                           rmse   land                                  9.8   
                                  ocean/sea-ice                         7.8   
                           mae    land                                  6.4   
                                  ocean/sea-ice                         5.8   

                                                                     \
climate                                                               
configuration                                   ML-corrected seed 2   
variable                   metric region                              
total_precipitation_rate   mean   land                          NaN   
                                  ocean/sea-ice                -0.1   
                           rmse   land                          0.8   
                                  ocean/sea-ice                 1.5   
                           mae    land                          0.5   
                                  ocean/sea-ice                 NaN   
surface_temperature        mean   land                          NaN   
                                  ocean/sea-ice                 0.0   
                           rmse   land                          NaN   
                                  ocean/sea-ice                 NaN   
                           mae    land                          NaN   
                                  ocean/sea-ice                 NaN   
net_surface_radiative_flux mean   land                          0.2   
                                  ocean/sea-ice                 NaN   
                           rmse   land                          7.3   
                                  ocean/sea-ice                 8.9   
                           mae    land                          5.1   
                                  ocean/sea-ice                 6.5   

                                                                            \
climate                                         Plus 4 K minus Unperturbed   
configuration                                                     Baseline   
variable                   metric region                                     
total_precipitation_rate   mean   land                                 NaN   
                                  ocean/sea-ice                       -0.0   
                           rmse   land                                 NaN   
                                  ocean/sea-ice                        1.9   
                           mae    land                                 NaN   
                                  ocean/sea-ice                  

In [8]:
def identical_sign(ds, dim):
    """Check if variables have the same sign for all values along a dimension"""
    return (ds > 0).all(dim) | (ds < 0).all(dim)

### To measure improvement of the RMSE we compute minus the percent deviation from the baseline

$$-100 \cdot \frac{M_{ML} - M_{baseline}}{M_{baseline}}$$

In the manuscript we only report cases where the percent improvement (or degredation) was statistically significant and of a consistent sign in all climates.

In [9]:
def compute_rmse_percent_improvement(metrics, significant, configuration):
    improvement = -(100 * (metrics.sel(configuration=configuration) - metrics.sel(configuration="Baseline")) / metrics.sel(configuration="Baseline")).drop("configuration")
    improvement_min = improvement.min("climate")
    improvement_max = improvement.max("climate")
    ds = xr.concat([improvement_min, improvement_max], dim=pd.Index(["min", "max"], name="bound")).sel(metric="rmse").drop("metric")
    ds = ds.where(significant.sel(metric="rmse").drop("metric").all("climate") & identical_sign(ds, "bound"))
    return ds[VARIABLES].sel(region=REGIONS).to_array().rename("metric").to_dataframe(dim_order=["variable", "region", "bound"]).unstack(level=-1)

In [10]:
compute_rmse_percent_improvement(mean_metrics_year_two, significant_year_two, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                       min   max
variable                   region                    
total_precipitation_rate   land             NaN   NaN
                           ocean/sea-ice    NaN   NaN
surface_temperature        land             NaN   NaN
                           ocean/sea-ice    NaN   NaN
net_surface_radiative_flux land             7.6  25.0
                           ocean/sea-ice    NaN   NaN

### To measure improvement of the bias we compute minus the percent deviation from the baseline for the absolute values

$$-100 \cdot \frac{|M_{ML}| - |M_{baseline}|}{|M_{baseline}|}$$

In the manuscript we only report cases where the percent improvement (or degredation) was statistically significant and of a consistent sign in all climates.

In [11]:
def compute_bias_percent_improvement(metrics, significant, configuration):
    improvement = -(100 * (np.abs(metrics.sel(configuration=configuration)) - np.abs(metrics.sel(configuration="Baseline"))) / np.abs(metrics.sel(configuration="Baseline"))).drop("configuration")
    improvement_min = improvement.min("climate")
    improvement_max = improvement.max("climate")
    ds = xr.concat([improvement_min, improvement_max], dim=pd.Index(["min", "max"], name="bound")).sel(metric="mean").drop("metric")
    ds = ds.where(significant.sel(metric="mean").drop("metric").all("climate") & identical_sign(ds, "bound"))
    return ds[VARIABLES].sel(region=REGIONS).to_array().rename("metric").to_dataframe(dim_order=["variable", "region", "bound"]).unstack(level=-1)

In [12]:
compute_bias_percent_improvement(mean_metrics_year_two, significant_year_two, "ML-corrected seed 2").round(decimals=1)

metric       
bound                                        min    max
variable                   region                      
total_precipitation_rate   land              NaN    NaN
                           ocean/sea-ice -6212.3 -172.8
surface_temperature        land              NaN    NaN
                           ocean/sea-ice     NaN    NaN
net_surface_radiative_flux land              NaN    NaN
                           ocean/sea-ice     NaN    NaN

### Impact of using the first year of the fine-resolution run as validation data

In [13]:
unmasked_year_one.round(decimals=1)

metric  \
climate                                         Unperturbed minus Minus 4 K   
configuration                                                      Baseline   
variable                   metric region                                      
total_precipitation_rate   mean   land                                 -0.0   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  0.9   
                                  ocean/sea-ice                         1.4   
                           mae    land                                  0.6   
                                  ocean/sea-ice                         0.9   
surface_temperature        mean   land                                  0.1   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  1.6   
                                  ocean/sea-ice                         0.3   
                           mae    land                                  1.1   
                                  ocean/sea-ice                         0.1   
net_surface_radiative_flux mean   land                                 -3.0   
                                  ocean/sea-ice                        -0.2   
                           rmse   land                                  9.6   
                                  ocean/sea-ice                         8.1   
                           mae    land                                  6.3   
                                  ocean/sea-ice                         6.1   

                                                                     \
climate                                                               
configuration                                   ML-corrected seed 2   
variable                   metric region                              
total_precipitation_rate   mean   land                          0.1   
                                  ocean/sea-ice                -0.1   
                           rmse   land                          0.9   
                                  ocean/sea-ice                 1.6   
                           mae    land                          0.6   
                                  ocean/sea-ice                 0.9   
surface_temperature        mean   land                         -0.2   
                                  ocean/sea-ice                 0.0   
                           rmse   land                          1.5   
                                  ocean/sea-ice                 0.3   
                           mae    land                          1.1   
                                  ocean/sea-ice                 0.1   
net_surface_radiative_flux mean   land                         -0.1   
                                  ocean/sea-ice                 0.3   
                           rmse   land                          7.7   
                                  ocean/sea-ice                 9.3   
                           mae    land                          5.3   
                                  ocean/sea-ice                 6.9   

                                                                            \
climate                                         Plus 4 K minus Unperturbed   
configuration                                                     Baseline   
variable                   metric region                                     
total_precipitation_rate   mean   land                                 0.1   
                                  ocean/sea-ice                        0.0   
                           rmse   land                                 1.0   
                                  ocean/sea-ice                        1.7   
                           mae    land                                 0.6   
                                  ocean/sea-ice                  

In [14]:
masked_year_one.round(decimals=1)

metric  \
climate                                         Unperturbed minus Minus 4 K   
configuration                                                      Baseline   
variable                   metric region                                      
total_precipitation_rate   mean   land                                  NaN   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  NaN   
                                  ocean/sea-ice                         1.4   
                           mae    land                                  NaN   
                                  ocean/sea-ice                         NaN   
surface_temperature        mean   land                                  NaN   
                                  ocean/sea-ice                        -0.0   
                           rmse   land                                  1.6   
                                  ocean/sea-ice                         NaN   
                           mae    land                                  NaN   
                                  ocean/sea-ice                         NaN   
net_surface_radiative_flux mean   land                                 -3.0   
                                  ocean/sea-ice                         NaN   
                           rmse   land                                  9.6   
                                  ocean/sea-ice                         8.1   
                           mae    land                                  6.3   
                                  ocean/sea-ice                         6.1   

                                                                     \
climate                                                               
configuration                                   ML-corrected seed 2   
variable                   metric region                              
total_precipitation_rate   mean   land                          NaN   
                                  ocean/sea-ice                -0.1   
                           rmse   land                          NaN   
                                  ocean/sea-ice                 1.6   
                           mae    land                          NaN   
                                  ocean/sea-ice                 NaN   
surface_temperature        mean   land                          NaN   
                                  ocean/sea-ice                 0.0   
                           rmse   land                          1.5   
                                  ocean/sea-ice                 NaN   
                           mae    land                          NaN   
                                  ocean/sea-ice                 NaN   
net_surface_radiative_flux mean   land                         -0.1   
                                  ocean/sea-ice                 NaN   
                           rmse   land                          7.7   
                                  ocean/sea-ice                 9.3   
                           mae    land                          5.3   
                                  ocean/sea-ice                 6.9   

                                                                            \
climate                                         Plus 4 K minus Unperturbed   
configuration                                                     Baseline   
variable                   metric region                                     
total_precipitation_rate   mean   land                                 NaN   
                                  ocean/sea-ice                        0.0   
                           rmse   land                                 1.0   
                                  ocean/sea-ice                        1.7   
                           mae    land                                 0.6   
                                  ocean/sea-ice                  

In [15]:
compute_rmse_percent_improvement(mean_metrics_year_one, significant_year_one, "ML-corrected seed 2").round(decimals=1)

metric      
bound                                       min   max
variable                   region                    
total_precipitation_rate   land             NaN   NaN
                           ocean/sea-ice    NaN   NaN
surface_temperature        land             NaN   NaN
                           ocean/sea-ice    NaN   NaN
net_surface_radiative_flux land             4.9  19.4
                           ocean/sea-ice    NaN   NaN

In [16]:
compute_bias_percent_improvement(mean_metrics_year_one, significant_year_one, "ML-corrected seed 2").round(decimals=1)

metric    
bound                                       min max
variable                   region                  
total_precipitation_rate   land             NaN NaN
                           ocean/sea-ice    NaN NaN
surface_temperature        land             NaN NaN
                           ocean/sea-ice    NaN NaN
net_surface_radiative_flux land             NaN NaN
                           ocean/sea-ice    NaN NaN